In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [55]:
from sklearn.model_selection import cross_val_score,train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error

In [61]:
from sklearn.linear_model import LinearRegression,RidgeCV,LassoCV
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor

In [54]:
def write_sumb(name,prediction):
    pd.DataFrame({'Id':range(1461,1461+1459),'SalePrice':prediction}).to_csv(name,index=False)

In [40]:
train = pd.read_csv('train_clean.csv')
test = pd.read_csv('test_clean.csv')

In [41]:
X_train,X_test,y_train,y_test = train_test_split(train.drop('SalePrice',axis=1),train.SalePrice,random_state=5,)

In [42]:
categorical_features = np.where(X_train.dtypes=='object')[0]
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

numeric_features = np.where((X_train.dtypes=='int')|(X_train.dtypes=='float'))[0]
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [58]:
X_train_prep = preprocessor.fit_transform(X_train)
X_test_prep = preprocessor.transform(X_test)
test_prep = preprocessor.transform(test)

In [59]:
linear = LinearRegression()
linear.fit(X_train_prep,y_train)
pred = linear.predict(test_prep)
write_sumb('linear.csv',pred)

In [62]:
lasso = LassoCV(random_state=5)
lasso.fit(X_train_prep,y_train)
pred = lasso.predict(test_prep)
write_sumb('lasso.csv',pred)

In [63]:
ridge = RidgeCV()
ridge.fit(X_train_prep,y_train)
pred = ridge.predict(test_prep)
write_sumb('ridge.csv',pred)

In [64]:
forest = RandomForestRegressor(random_state=5)
forest.fit(X_train_prep,y_train)
pred = forest.predict(test_prep)
write_sumb('forest.csv',pred)

In [65]:
gbr = GradientBoostingRegressor(random_state=5)
gbr.fit(X_train_prep,y_train)
pred = gbr.predict(test_prep)
write_sumb('gbr.csv',pred)

In [68]:
import tensorflow as tf
import tensorflow.keras

In [116]:
X_train_ = X_train_prep.toarray()
X_test_ = X_test_prep.toarray()
test_ = test_prep.toarray()

In [170]:
X_train_tf = tf.convert_to_tensor(X_train_,dtype='float64')
X_test_tf = tf.convert_to_tensor(X_test_,dtype='float64')
test_tf = tf.convert_to_tensor(test_,dtype='float64')
y_train_tf = tf.convert_to_tensor(y_train,dtype='float64')

In [171]:
y_train_tf

<tf.Tensor: shape=(1095,), dtype=float64, numpy=array([204000., 185000., 143000., ...,  91000., 143900., 129000.])>

In [172]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Flatten(dtype='float64'),
        tf.keras.layers.Dense(128,activation='relu',dtype='float64'),
        tf.keras.layers.Dense(1,dtype='float64')
    ]
)
model.compile(optimizer='adam',loss=tf.keras.losses.MeanSquaredError(),metrics=tf.keras.metrics.mean_squared_error)

In [177]:
model.fit(X_train_tf,y_train_tf,epochs=1000)

Epoch 1/1000
35/35 [==============================] - 0s 1ms/step - loss: 38951649280.0000 - mean_squared_error: 38951649280.0000
Epoch 2/1000
35/35 [==============================] - 0s 886us/step - loss: 38938529792.0000 - mean_squared_error: 38938529792.0000
Epoch 3/1000
35/35 [==============================] - 0s 1ms/step - loss: 38918799360.0000 - mean_squared_error: 38918799360.0000
Epoch 4/1000
35/35 [==============================] - 0s 867us/step - loss: 38891905024.0000 - mean_squared_error: 38891905024.0000
Epoch 5/1000
35/35 [==============================] - 0s 1ms/step - loss: 38858018816.0000 - mean_squared_error: 38858018816.0000
Epoch 6/1000
35/35 [==============================] - 0s 837us/step - loss: 38816522240.0000 - mean_squared_error: 38816522240.0000
Epoch 7/1000
35/35 [==============================] - 0s 793us/step - loss: 38767443968.0000 - mean_squared_error: 38767443968.0000
Epoch 8/1000
35/35 [==============================] - 0s 807us/step - loss: 387105

In [221]:
pred = model.predict(test_tf)
pred = np.array(pred).ravel()
write_sumb('keras.csv',pred)

In [253]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Flatten(dtype='float64'),
        tf.keras.layers.Dense(256,activation='relu',dtype='float64'),
        tf.keras.layers.BatchNormalization(dtype='float64'),
        tf.keras.layers.Dropout(0.2,dtype='float64'),
        tf.keras.layers.Dense(128,activation='relu',dtype='float64'),
        tf.keras.layers.BatchNormalization(dtype='float64'),
        tf.keras.layers.Dropout(0.2,dtype='float64'),
        tf.keras.layers.Dense(32,activation='relu',dtype='float64'),
        tf.keras.layers.Dense(1,dtype='float64')
    ]
)
#sgd = tf.keras.optimizers.SGD(learning_rate=0.01)
model.compile(optimizer='adam',loss=tf.keras.losses.MeanSquaredError(),metrics=tf.keras.metrics.mean_squared_error)

In [254]:
model.fit(X_train_tf,y_train_tf,epochs=500,batch_size=16)

Epoch 1/500
69/69 [==============================] - 0s 2ms/step - loss: 38959726592.0000 - mean_squared_error: 38959726592.0000
Epoch 2/500
69/69 [==============================] - 0s 2ms/step - loss: 38944350208.0000 - mean_squared_error: 38944350208.0000
Epoch 3/500
69/69 [==============================] - 0s 2ms/step - loss: 38901805056.0000 - mean_squared_error: 38901805056.0000
Epoch 4/500
69/69 [==============================] - 0s 2ms/step - loss: 38816067584.0000 - mean_squared_error: 38816067584.0000
Epoch 5/500
69/69 [==============================] - 0s 2ms/step - loss: 38670319616.0000 - mean_squared_error: 38670319616.0000
Epoch 6/500
69/69 [==============================] - 0s 2ms/step - loss: 38446321664.0000 - mean_squared_error: 38446321664.0000
Epoch 7/500
69/69 [==============================] - 0s 2ms/step - loss: 38132076544.0000 - mean_squared_error: 38132072448.0000
Epoch 8/500
69/69 [==============================] - 0s 1ms/step - loss: 37690159104.0000 - mean_

In [255]:
pred = model.predict(test_tf)
pred = np.array(pred).ravel()
write_sumb('keras.csv',pred)

In [256]:
pred

array([127141.41522967, 198232.39744689, 192388.46632104, ...,
       171863.46881001, 118952.26567613, 212171.06678593])